In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
import scipy.stats as st

In [5]:
movie_metadata_path = "C:/Users/sholl/OneDrive/Desktop/GIT/Project-1/Resources/imdb_top250_movies.csv"
movie_data_complete = pd.read_csv(movie_metadata_path)
movie_data_complete

,Unnamed: 0,Num,Title,Year,Released,Runtime,Genre,Director,Writer,Actors,...,Awards,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website
0,0,1,The Shawshank Redemption,1994,14-Oct-94,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",...,Nominated for 7 Oscars. Another 19 wins & 30 n...,80.0,9.3,"1,825,626",tt0111161,movie,27-Jan-98,NaN,Columbia Pictures,NaN
1,1,2,The Godfather,1972,24-Mar-72,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",...,Won 3 Oscars. Another 23 wins & 27 nominations.,100.0,9.2,"1,243,444",tt0068646,movie,9-Oct-01,NaN,Paramount Pictures,http://www.thegodfather.com
2,2,3,The Godfather: Part II,1974,20-Dec-74,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",...,Won 6 Oscars. Another 10 wins & 20 nominations.,85.0,9.0,"856,870",tt0071562,movie,24-May-05,NaN,Paramount Pictures,http://www.thegodfather.com/
3,3,4,The Dark Knight,2008,18-Jul-08,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",...,Won 2 Oscars. Another 151 wins & 153 nominations.,82.0,9.0,"1,802,351",tt0468569,movie,9-Dec-08,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/
4,4,5,12 Angry Men,1957,1-Apr-57,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",...,Nominated for 3 Oscars. Another 16 wins & 8 no...,96.0,8.9,"494,215",tt0050083,movie,6-Mar-01,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,245,246,The Lost Weekend,1945,1-Jan-46,101 min,"Drama, Film-Noir",Billy Wilder,"Charles R. Jackson (from the novel by), Charle...","Ray Milland, Jane Wyman, Phillip Terry, Howard...",...,Won 4 Oscars. Another 12 wins & 3 nominations.,NaN,8.0,"26,834",tt0037884,movie,6-Feb-01,NaN,Paramount Pictures,NaN
246,246,247,Short Term 12,2013,23-Aug-13,96 min,Drama,Destin Daniel Cretton,Destin Daniel Cretton,"Brie Larson, John Gallagher Jr., Stephanie Bea...",...,35 wins & 70 nominations.,82.0,8.0,"62,348",tt2370248,movie,14-Jan-14,NaN,Cinedigm,http://shortterm12.com
247,247,248,His Girl Friday,1940,18-Jan-40,92 min,"Comedy, Drama, Romance",Howard Hawks,"Charles Lederer (screen play), Ben Hecht (from...","Cary Grant, Rosalind Russell, Ralph Bellamy, G...",...,2 wins.,NaN,8.0,"43,551",tt0032599,movie,28-Dec-04,NaN,Columbia Pictures,NaN
248,248,249,The Straight Story,1999,3-Nov-99,112 min,"Biography, Drama",David Lynch,"John Roach, Mary Sweeney","Sissy Spacek, Jane Galloway Heitz, Joseph A. C...",...,Nominated for 1 Oscar. Another 14 wins & 37 no...,86.0,8.0,"67,045",tt0166896,movie,7-Nov-00,NaN,Buena Vista Pictures,http://disney.go.com/DisneyPictures/straightst...
